# First Load the dataset and create vocab

In [1]:
from gpt_tf import VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN
from gpt_tf import TransformerBlock, TokenAndPositionEmbedding, causal_attention_mask

In [2]:
import numpy as np
import pathlib
from tensorflow.keras import layers, models

In [3]:
# Load the full dataset
import pathlib
import json

datasets_folder = pathlib.Path(r"C:\Users\amrul\programming\deep_learning\dl_projects\Generative_Deep_Learning_2nd_Edition\data")
wine_review_filepath=datasets_folder/"wine_reviews"/"winemag-data-130k-v2.json"
with open(str(wine_review_filepath)) as json_data:
    wine_data = json.load(json_data)

In [4]:
# Filter the dataset
filtered_data = [
    "wine review : "
    + x["country"]
    + " : "
    + x["province"]
    + " : "
    + x["variety"]
    + " : "
    + x["description"]
    for x in wine_data
    if x["country"] is not None
    and x["province"] is not None
    and x["variety"] is not None
    and x["description"] is not None
]

In [5]:
import re
import string

# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]

In [6]:
import tensorflow as tf
from gpt_tf import BATCH_SIZE
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [7]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [8]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

# Let's load the trained model and generate texts

In [9]:
saved_model_path=pathlib.Path(r"C:\Users\amrul\programming\deep_learning\dl_projects\Generative_Deep_Learning_2nd_Edition\notebooks\09_transformer\gpt\models\gpt")
gpt = models.load_model(str(saved_model_path))

In [10]:
gpt.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddi  (None, None, 256)         2580480   
 ng (TokenAndPositionEmbedd                                      
 ing)                                                            
                                                                 
 transformer_block (Transfo  ((None, None, 256),       658688    
 rmerBlock)                   (None, 2, None, None))             
                                                                 
 dense_2 (Dense)             (None, None, 10000)       2570000   
                                                                 
Total params: 5809168 (22.16 MB)
Trainable params: 5809168 (22.16 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [11]:
from gpt_tf import TextGenerator

In [13]:
# let's import TextGenerator, initialize it and set its model
from gpt_tf import TextGenerator
text_generator=TextGenerator(vocab, gpt)


In [15]:
info=text_generator.generate("wine review : us : chardaney",MAX_LEN,0.8)


generated text:
wine review : us : chardaney : red blend : a blend of sangiovese , cabernet sauvignon and petit verdot , this is a thick , dense and tannic wine that delivers a deep core of black cherry , spice and peppercorn that carries a hint of vanilla . smooth , firm and velvety on the palate , with a hint of fresh greens . 



In [16]:
from gpt_tf import print_probs

print_probs(info,vocab)

::   	98.34%
valley:   	0.55%
hills:   	0.47%
-:   	0.18%
new:   	0.09%
--------



chardonnay:   	20.96%
cabernet:   	17.33%
pinot:   	13.89%
red:   	10.95%
merlot:   	8.85%
--------



blend:   	99.99%
wine:   	0.0%
::   	0.0%
[UNK]:   	0.0%
bordeaux:   	0.0%
--------



::   	100.0%
-:   	0.0%
grosso:   	0.0%
of:   	0.0%
blend:   	0.0%
--------



this:   	49.13%
a:   	15.43%
the:   	7.27%
aromas:   	1.62%
made:   	1.51%
--------



...

of:   	100.0%
at:   	0.0%
and:   	0.0%
to:   	0.0%
from:   	0.0%
--------



chocolate:   	19.24%
tobacco:   	5.59%
black:   	5.48%
leather:   	4.86%
dark:   	3.17%
--------



herb:   	21.91%
acidity:   	15.42%
tobacco:   	9.81%
-:   	9.41%
herbs:   	6.61%
--------



.:   	36.91%
and:   	34.93%
,:   	16.91%
on:   	2.03%
that:   	1.57%
--------



:   	95.96%
it:   	0.95%
drink:   	0.8%
the:   	0.61%
a:   	0.47%
--------

